![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/ER_RXNORM.ipynb)

## **Resolve Drugs using the RxNorm taxonomy**

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload license_keys.json to the folder that opens. Otherwise, you can look at the example outputs at the bottom of the notebook.

## **Colab Setup**

In [ ]:
import json
import os

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

## **Install dependencies**

In [3]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

## **Import dependencies into Python and start the Spark session**

In [4]:
import json
import os

from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SparkSession

import sparknlp
import sparknlp_jsl

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.pretrained import ResourceDownloader
from pyspark.sql import functions as F

from sparknlp_display import EntityResolverVisualizer

import pandas as pd

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

import string
import numpy as np

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(secret = SECRET, params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 3.4.4
Spark NLP_JSL Version : 3.5.2


### **🔎 For about models**

📌 **sbiobertresolve_rxnorm**--> *This model maps clinical entities and concepts (like drugs/ingredients) to RxNorm codes using sbiobert_base_cased_mli Sentence Bert Embeddings.*


📌 **sbiobertresolve_rxnorm_disposition** --> *This model maps medication entities (like drugs/ingredients) to RxNorm codes and their dispositions using sbiobert_base_cased_mli Sentence Bert Embeddings.sbiobertresolve_rxnorm_disposition resolver model must be used with sbiobert_base_cased_mli as embeddings ner_posology as NER model. DRUG set in .setWhiteList().*

📌 **sbiobertresolve_rxnorm_augmented_re** --> *This model maps clinical entities and concepts (like drugs/ingredients) to RxNorm codes without specifying the relations between the entities (relations are calculated on the fly inside the annotator) using sbiobert_base_cased_mli Sentence Bert Embeddings (EntityChunkEmbeddings). Embeddings used in this model are calculated with following weights : {"DRUG": 0.8, "STRENGTH": 0.2, "ROUTE": 0.2, "FORM": 0.2} . EntityChunkEmbeddings with those weights are required in the pipeline to get best result.*



### **🔎 Helper Function**


In [5]:
# returns spark df resolution results

def get_codes_from_df(result_df, chunk, output_col, hcc= False):
    
    
    if hcc:
        
        df = result_df.select(F.explode(F.arrays_zip(chunk+'.result', 
                                                           chunk+'.metadata', 
                                                           output_col+'.result', 
                                                           output_col+'.metadata')).alias("cols")) \
                                     .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                                             F.expr("cols['0']").alias("ner_chunk"),
                                             F.expr("cols['1']['entity']").alias("entity"), 
                                             F.expr("cols['2']").alias("icd10_code"),
                                             F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                             F.expr("cols['3']['all_k_resolutions']").alias("resolutions"),
                                             F.expr("cols['3']['all_k_aux_labels']").alias("hcc_list")).toPandas()



        codes = []
        resolutions = []
        hcc_all = []

        for code, resolution, hcc in zip(df['all_codes'], df['resolutions'], df['hcc_list']):

            codes.append(code.split(':::'))
            resolutions.append(resolution.split(':::'))
            hcc_all.append(hcc.split(":::"))

        df['all_codes'] = codes  
        df['resolutions'] = resolutions
        df['hcc_list'] = hcc_all
        
    else:
                       
        df = result_df.select(F.explode(F.arrays_zip(chunk+'.result', 
                                                           chunk+'.metadata', 
                                                           output_col+'.result', 
                                                           output_col+'.metadata')).alias("cols")) \
                                     .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                                             F.expr("cols['0']").alias("ner_chunk"),
                                             F.expr("cols['1']['entity']").alias("entity"), 
                                             F.expr("cols['2']").alias(f"{output_col}"),
                                             F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                             F.expr("cols['3']['all_k_resolutions']").alias("resolutions")).toPandas()



        codes = []
        resolutions = []

        for code, resolution in zip(df['all_codes'], df['resolutions']):

            codes.append(code.split(':::'))
            resolutions.append(resolution.split(':::'))

        df['all_codes'] = codes  
        df['resolutions'] = resolutions
        
    
    return df

# **📌 "sbiobertresolve_rxnorm" model**

### **🔎Define Spark NLP pipeline**

In [5]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")
'''
drug_normalizer = DrugNormalizer() \
      .setInputCols("document") \
      .setOutputCol("drug_normalized") \
      .setPolicy('all')'''

sentenceDetector = SentenceDetectorDLModel.pretrained()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")\

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

posology_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(['DRUG'])

c2doc = Chunk2Doc()\
      .setInputCols("ner_chunk")\
      .setOutputCol("ner_chunk_doc") 

sbert_embedder = BertSentenceEmbeddings\
      .pretrained("sbiobert_base_cased_mli",'en','clinical/models')\
      .setInputCols(["ner_chunk_doc"])\
      .setOutputCol("sbert_embeddings")

rxnorm_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm","en", "clinical/models") \
      .setInputCols(["ner_chunk_doc", "sbert_embeddings"]) \
      .setOutputCol("rxnorm_code")\
      .setDistanceFunction("EUCLIDEAN")

resolver_pipeline = Pipeline(
    stages = [
        documentAssembler,
        #drug_normalizer,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        posology_ner,
        ner_converter,
        c2doc,
        sbert_embedder,
        rxnorm_resolver])

data_ner = spark.createDataFrame([[""]]).toDF("text")

model = resolver_pipeline.fit(data_ner)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_posology download started this may take some time.
[OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_rxnorm download started this may take some time.
[OK!]


In [6]:
sample_text = """The patient is a 40-year-old white male who presents with a chief complaint of "chest pain". The patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. He has been advised Aspirin 81 milligrams QDay. Humulin N. insulin 50 units in a.m. Hydrochlorothiazide 50 mg QDay. Nitroglycerin 1/150 sublingually PRN chest pain."""

clinical_note_df = spark.createDataFrame([[sample_text]]).toDF("text")

rxnorm_result = model.transform(clinical_note_df)

In [7]:
res_pd = get_codes_from_df(rxnorm_result, 'ner_chunk', 'rxnorm_code', hcc=False)

In [8]:
res_pd.head(10)

,sent_id,ner_chunk,entity,rxnorm_code,all_codes,resolutions
0,3,Aspirin,DRUG,1191,"[1191, 405403, 218266, 215448, 215568, 1154070, 215803, 1154069, 215436, 216110, 215444, 202547, 724441, 1535483, 375972, 370941, 1182756, 1186642, 215452, 1294937, 1154068, 215434, 1363743, 379497, 611]","[aspirin, ysp aspirin, med aspirin, aspirin-antacid, bayer aspirin, aspirin pill, ctd aspirin, aspirin oral product, buffered aspirin, coated aspirin, aspirin lo-dose, empirin, anacin aspirin regimen, aspirin oral strip, aspirin oral solution, aspirin injectable solution, med aspirin oral product, ysp aspirin oral product, aspirtab, aspirin chewable product, aspirin oral liquid product, aspir-trin, flanax aspirin, aspirin oral powder, aluminum aspirin]"
1,4,Humulin N,DRUG,92880,"[92880, 218686, 92879, 261588, 92881, 1372744, 1720602, 1723758, 539898, 892767, 7550, 227449, 657414, 218621, 218741, 1099270, 218644, 542008, 152098, 218589, 31538, 89721, 218564, 134802, 353501]","[humulin n, neotricin hc, humulin l, nabi-hb, humulin r, humulin, nucala, ninlaro, nudal hd, nauzene, nubain, navane, nariz hc, nasarel, nilandron, nulecit, navogan, numonyl, natrilix, nallpen, naringin, nitromersol, naftin, naropin, neulasta]"
2,4,insulin,DRUG,5856,"[5856, 484319, 139825, 1740938, 274783, 86009, 1605101, 5924, 1670007, 314683, 542029, 51428, 92972, 400008, 1086383, 9627, 221110, 1160696, 1157461, 485206, 485207, 84959, 631657, 1803887, 202559]","[insulin, insulin detemir, insulin detemir, insulin argine, insulin glargine, insulin lispro, insulin isophane, inulin, insulin degludec, lente insulin, human, insulase, insulin aspart, human, insta-glucose, insulin glulisine, human, enfamil glucose, secretin, ultralente insulin, human, insulin detemir injectable product, insulin lispro injectable product, insulin, glulisine, human, insulin, glulisine, human, imiglucerase, insulin human, rdna origin, adlyxin, imuran]"
3,5,Hydrochlorothiazide,DRUG,5487,"[5487, 1162786, 2396, 91217, 203165, 82027, 1162785, 1360720, 1162787, 91135, 297716, 235450, 1731061, 2197879, 1311672, 370637, 236679, 48928, 235752, 235726, 154987, 292726, 221126, 235765, 236297]","[hydrochlorothiazide, hydrochlorothiazide oral product, chlorothiazide, chlorothiazide sodium, thioridazine hydrochloride, hydralazine hydrochloride, hydrochlorothiazide oral liquid product, thonzylamine hydrochloride, hydrochlorothiazide pill, thiothixene hydrochloride, phendimetrazine hydrochloride, tolazoline hydrochloride, chlorothiazide injection, pitolisant hydrochloride, indeloxazine hydrochloride, hydrochlorothiazide oral solution, morazone hydrochloride, detomidine hydrochloride, bepridil hydrochloride, isothipendyl hydrochloride, hydroxyzine hydrochloride, phenylhydrazine hydrochloride, moricizine hydrochloride, benserazide hydrochloride, chlorproethazine hydrochloride]"
4,6,Nitroglycerin,DRUG,4917,"[4917, 360398, 1868493, 1159827, 1159829, 379759, 1159830, 1159826, 377109, 218753, 1295349, 1295590, 202516, 202525, 7393, 705128, 7441, 151175, 218750, 379731, 1159828, 218734, 378657, 7476, 202508]","[nitroglycerin, able brand of nitroglycerin, nitroglycerin injection, nitroglycerin mucosal product, nitroglycerin oral product, nitromin brand of nitroglycerin, nitroglycerin pill, nitroglycerin injectable product, nitroglycerin injectable solution, nitropress, nitroglycerin chewable product, nitroglycerin buccal product, nitro-bid, nitrostat, niacin, nitroglycerin mucosal spray, nitrendipine, nitrocine, nitrogard, nitroglycerin injectable solution [nitrocine], nitroglycerin oral liquid product, nicotrol, nitroglycerin oral spray, nitroprusside, nitro-dur]"


In [9]:
light_model = LightPipeline(model)
light_result = light_model.fullAnnotate(sample_text)

er_vis = EntityResolverVisualizer()

er_vis.display(light_result[0],
               label_col='ner_chunk',
               resolution_col = 'rxnorm_code',
               document_col='document'
               )

# **📌 "sbiobertresolve_rxnorm_disposition" model**

### **🔎Define Spark NLP pipeline**

In [10]:
rxnorm_resolver = SentenceEntityResolverModel.pretrained('sbiobertresolve_rxnorm_disposition', "en", "clinical/models") \
        .setInputCols(["ner_chunk", "sbert_embeddings"]) \
        .setOutputCol("rxnorm_code")\
        .setDistanceFunction("EUCLIDEAN")

resolver_pipeline = Pipeline(stages = [
              documentAssembler,
              sentenceDetector,
              tokenizer,
              word_embeddings,
              posology_ner,
              ner_converter,
              c2doc,
              sbert_embedder,
              rxnorm_resolver])

data_ner = spark.createDataFrame([[""]]).toDF("text")
model = resolver_pipeline.fit(data_ner)
light_model = LightPipeline(model)

sbiobertresolve_rxnorm_disposition download started this may take some time.
[OK!]


In [11]:
sample_text = """The patient is a 40-year-old white male who presents with a chief complaint of "chest pain". The patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. He has been advised Aspirin 81 milligrams QDay. Humulin N. insulin 50 units in a.m. Hydrochlorothiazide 50 mg QDay. Nitroglycerin 1/150 sublingually PRN chest pain."""


clinical_note_df = spark.createDataFrame([[sample_text]]).toDF("text")
rxnorm_sdf = model.transform(clinical_note_df)

In [12]:
res_pd = get_codes_from_df(rxnorm_sdf, 'ner_chunk', 'rxnorm_code', hcc=False)

In [13]:
res_pd.head(10)

,sent_id,ner_chunk,entity,rxnorm_code,all_codes,resolutions
0,3,Aspirin,DRUG,1191,"[1191, 405403, 218266, 215448, 215568, 1154070, 215803, 1154069, 215436, 216110, 215444, 202547, 724441, 1535483, 375972, 370941, 1182756, 1186642, 215452, 1294937, 1154068, 215434, 1363743, 379497, 611]","[aspirin, ysp aspirin, med aspirin, aspirin-antacid, bayer aspirin, aspirin pill, ctd aspirin, aspirin oral product, buffered aspirin, coated aspirin, aspirin lo-dose, empirin, anacin aspirin regimen, aspirin oral strip, aspirin oral solution, aspirin injectable solution, med aspirin oral product, ysp aspirin oral product, aspirtab, aspirin chewable product, aspirin oral liquid product, aspir-trin, flanax aspirin, aspirin oral powder, aluminum aspirin]"
1,4,Humulin N,DRUG,92880,"[92880, 218686, 92879, 261588, 92881, 1372744, 1720602, 1723758, 539898, 892767, 7550, 227449, 657414, 218621, 218741, 1099270, 218644, 542008, 152098, 218589, 31538, 89721, 218564, 134802, 353501]","[humulin n, neotricin hc, humulin l, nabi-hb, humulin r, humulin, nucala, ninlaro, nudal hd, nauzene, nubain, navane, nariz hc, nasarel, nilandron, nulecit, navogan, numonyl, natrilix, nallpen, naringin, nitromersol, naftin, naropin, neulasta]"
2,4,insulin,DRUG,5856,"[5856, 484319, 139825, 1740938, 274783, 86009, 1605101, 5924, 1670007, 314683, 542029, 51428, 92972, 400008, 1086383, 9627, 221110, 1160696, 1157461, 485206, 485207, 84959, 631657, 1803887, 202559]","[insulin, insulin detemir, insulin detemir, insulin argine, insulin glargine, insulin lispro, insulin isophane, inulin, insulin degludec, lente insulin, human, insulase, insulin aspart, human, insta-glucose, insulin glulisine, human, enfamil glucose, secretin, ultralente insulin, human, insulin detemir injectable product, insulin lispro injectable product, insulin, glulisine, human, insulin, glulisine, human, imiglucerase, insulin human, rdna origin, adlyxin, imuran]"
3,5,Hydrochlorothiazide,DRUG,5487,"[5487, 1162786, 2396, 91217, 203165, 82027, 1162785, 1360720, 1162787, 91135, 297716, 235450, 1731061, 2197879, 1311672, 370637, 236679, 48928, 235752, 235726, 154987, 292726, 221126, 235765, 236297]","[hydrochlorothiazide, hydrochlorothiazide oral product, chlorothiazide, chlorothiazide sodium, thioridazine hydrochloride, hydralazine hydrochloride, hydrochlorothiazide oral liquid product, thonzylamine hydrochloride, hydrochlorothiazide pill, thiothixene hydrochloride, phendimetrazine hydrochloride, tolazoline hydrochloride, chlorothiazide injection, pitolisant hydrochloride, indeloxazine hydrochloride, hydrochlorothiazide oral solution, morazone hydrochloride, detomidine hydrochloride, bepridil hydrochloride, isothipendyl hydrochloride, hydroxyzine hydrochloride, phenylhydrazine hydrochloride, moricizine hydrochloride, benserazide hydrochloride, chlorproethazine hydrochloride]"
4,6,Nitroglycerin,DRUG,4917,"[4917, 360398, 1868493, 1159827, 1159829, 379759, 1159830, 1159826, 377109, 218753, 1295349, 1295590, 202516, 202525, 7393, 705128, 7441, 151175, 218750, 379731, 1159828, 218734, 378657, 7476, 202508]","[nitroglycerin, able brand of nitroglycerin, nitroglycerin injection, nitroglycerin mucosal product, nitroglycerin oral product, nitromin brand of nitroglycerin, nitroglycerin pill, nitroglycerin injectable product, nitroglycerin injectable solution, nitropress, nitroglycerin chewable product, nitroglycerin buccal product, nitro-bid, nitrostat, niacin, nitroglycerin mucosal spray, nitrendipine, nitrocine, nitrogard, nitroglycerin injectable solution [nitrocine], nitroglycerin oral liquid product, nicotrol, nitroglycerin oral spray, nitroprusside, nitro-dur]"


In [14]:
light_result = light_model.fullAnnotate(sample_text)

er_vis = EntityResolverVisualizer()

er_vis.display(light_result[0],
               label_col='ner_chunk',
               resolution_col = 'rxnorm_code',
               document_col='document'
               )

# **📌 "sbiobertresolve_rxnorm_augmented_re" model**

***🔎Check here for more information.***

-->https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/3.2.Sentence_Entity_Resolvers_with_EntityChunkEmbeddings.ipynb

### **🔎Define Spark NLP pipeline**

In [6]:
documenter = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentence_detector = SentenceDetector() \
    .setInputCols("document") \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols("sentence") \
    .setOutputCol("token")

embeddings = WordEmbeddingsModel() \
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

posology_ner_model = MedicalNerModel()\
    .pretrained("ner_posology_large", "en", "clinical/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols("sentence", "token", "ner")\
    .setOutputCol("ner_chunk")

pos_tager = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models")\
    .setInputCols("sentence", "token")\
    .setOutputCol("pos_tag")

dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos_tag", "token"])\
    .setOutputCol("dependencies")

drug_chunk_embeddings = EntityChunkEmbeddings()\
    .pretrained("sbiobert_base_cased_mli","en","clinical/models")\
    .setInputCols(["ner_chunk", "dependencies"])\
    .setOutputCol("drug_chunk_embeddings")\
    .setMaxSyntacticDistance(5)\
    .setTargetEntities({"DRUG": ["STRENGTH", "ROUTE", "FORM"]})\
    .setEntityWeights({"DRUG": 0.8, "STRENGTH": 0.2, "ROUTE": 0.2, "FORM": 0.2})

rxnorm_re = SentenceEntityResolverModel\
      .pretrained("sbiobertresolve_rxnorm_augmented_re", "en","clinical/models")\
      .setInputCols(["ner_chunk","drug_chunk_embeddings"])\
      .setOutputCol("rxnorm_code")\
      .setDistanceFunction("EUCLIDEAN")

rxnorm_pipeline_re = Pipeline(
    stages = [
        documenter,
        sentence_detector,
        tokenizer,
        embeddings,
        posology_ner_model,
        ner_converter,
        pos_tager,
        dependency_parser,
        drug_chunk_embeddings,
        rxnorm_re
        ])

emty_data = spark.createDataFrame([[""]]).toDF("text")
model = rxnorm_pipeline_re.fit(emty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_posology_large download started this may take some time.
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]
sbiobert_base_cased_mli download started this may take some time.
[OK!]
sbiobertresolve_rxnorm_augmented_re download started this may take some time.
[OK!]


In [7]:
sample_text = """The patient is a 40-year-old white male who presents with a chief complaint of "chest pain". The patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. He has been advised Aspirin 81 milligrams QDay. Humulin N. insulin 50 units in a.m. Hydrochlorothiazide 50 mg QDay. Nitroglycerin 1/150 sublingually PRN chest pain."""


clinical_note_df = spark.createDataFrame([[sample_text]]).toDF("text")
rxnorm_sdf = model.transform(clinical_note_df)

In [12]:
rxnorm_sdf.select('rxnorm_code.metadata').show(truncate = 200)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                metadata|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[{all_k_results -> 1535484:::434451:::247138:::315431:::243670:::252857:::318272:::211830:::211832:::404658, all_k_distances -> 3.2895:::3.3071:::3.3071:::3.3071:::3.3071:::3.3071:::3.3071:::3.3071...|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
rxnorm_df = rxnorm_sdf.select(F.explode(F.arrays_zip(rxnorm_sdf.drug_chunk_embeddings.result, 
                                                           rxnorm_sdf.drug_chunk_embeddings.begin,
                                                           rxnorm_sdf.drug_chunk_embeddings.end,
                                                           rxnorm_sdf.rxnorm_code.result, 
                                                           rxnorm_sdf.rxnorm_code.metadata,
                                                           rxnorm_sdf.drug_chunk_embeddings.metadata).alias("col")))\
                .select( 
                        F.expr("col['0']").alias("chunk"),
                        F.expr("col['1']").alias("begin"),
                        F.expr("col['2']").alias("end"),
                        F.expr("col['5']['target_entity']").alias("entity_type"),
                        F.expr("col['3']").alias("RxNorm_code"),
                        F.expr("col['4']['all_k_resolutions']").alias("all_k_resolutions") ,
                        F.expr("col['4']['all_k_results']").alias("all_k_codes"),
                        F.expr("col['4']['all_k_aux_labels']").alias("all_k_labels")
                        ).toPandas()

In [15]:
rxnorm_df

,chunk,begin,end,entity_type,RxNorm_code,all_k_resolutions,all_k_codes,all_k_labels
0,Aspirin 81 milligrams,306,326,DRUG,1535484,aspirin 81 MG Oral Film:::aspirin 130 MG Oral Lozenge:::aspirin 850 MG Oral Powder:::aspirin 81 MG:::aspirin 81 MG Oral Tablet:::aspirin 81 MG Oral Capsule:::aspirin 81 MG Chewable Tablet:::aspirin 81 MG Chewable Tablet [Med Aspirin]:::aspirin 81 MG Chewable Tablet [St. Joseph Aspirin]:::aspirin 81 MG Delayed Release Oral Capsule [YSP Aspirin],1535484:::434451:::247138:::315431:::243670:::252857:::318272:::211830:::211832:::404658,Clinical Drug:::Clinical Drug:::Clinical Drug:::Clinical Drug Comp:::Clinical Drug:::Clinical Drug:::Clinical Drug:::Branded Drug:::Branded Drug:::Branded Drug
1,Humulin,334,340,DRUG,1359720,"insulin isophane, human 70 UNT/ML / insulin, regular, human 30 UNT/ML Prefilled Syringe [Humulin]:::insulin isophane, human 70 UNT/ML / insulin, regular, human 30 UNT/ML Injectable Suspension [Humulin]:::insulin isophane, human 70 UNT/ML / insulin, regular, human 30 UNT/ML Pen Injector [Humulin]:::insulin isophane, human 80 UNT/ML / insulin, regular, human 20 UNT/ML Injectable Suspension [Humulin]:::insulin isophane / insulin, regular, human Pen Injector [Humulin]:::insulin isophane / insulin, regular, human Prefilled Syringe [Humulin]:::insulin isophane, human 60 UNT/ML / insulin, regular, human 40 UNT/ML Injectable Suspension [Humulin]:::3 ML insulin isophane, human 70 UNT/ML / insulin, regular, human 30 UNT/ML Pen Injector [Humulin]:::1.5 ML insulin isophane, human 70 UNT/ML / insulin, regular, human 30 UNT/ML Prefilled Syringe [Humulin]:::insulin, regular, human Pen Injector [Humulin R]:::insulin isophane, human 100 UNT/ML Injectable Suspension [Humulin I]:::lente insulin, human Injectable Suspension [Humulin L]:::lente insulin, human 100 UNT/ML Injectable Suspension [Humulin L]:::ultralente insulin, human 100 UNT/ML Injectable Suspension [Humulin U Ultralente]:::insulin isophane, human 90 UNT/ML / insulin, regular, human 10 UNT/ML Injectable Suspension [Humulin]",1359720:::106892:::1654858:::106900:::1654855:::847188:::106901:::847189:::847343:::1731316:::150979:::365694:::311019:::314045:::106899,Branded Drug:::Branded Drug:::Branded Drug:::Branded Drug:::Branded Drug Form:::Branded Drug Form:::Branded Drug:::Quant Branded Drug:::Quant Branded Drug:::Branded Drug Form:::Branded Drug:::Branded Drug Form:::Branded Drug:::Branded Drug:::Branded Drug
2,insulin,345,351,DRUG,2179743,"insulin, regular, human Injection:::insulin, regular, human 1 UNT/ML Injection:::insulin, regular, beef-pork 100 UNT/ML Injectable Solution:::insulin, regular, beef-pork 80 UNT/ML Injectable Solution:::insulin, regular, human Inhalant Product:::insulin, regular, pork 100 UNT/ML Injectable Solution:::insulin, regular, human 40 UNT/ML Injectable Solution:::insulin, regular, human 4 UNT/ML:::insulin, regular, beef-pork 40 UNT/ML:::insulin, regular, human 8 UNT Inhalation Powder:::insulin, regular, pork 40 UNT/ML Injectable Solution:::insulin, regular, human Pen Injector:::insulin, regular, human 4 UNT Inhalation Powder:::insulin, regular, pork 30 UNT/ML:::insulin, regular, pork 80 UNT/ML Injectable Solution:::insulin, regular, pork 500 UNT/ML:::insulin, regular, beef 80 UNT/ML:::insulin, pork:::insulin, regular, pork:::insulin, regular, human 500 UNT/ML:::insulin, regular, human 8 UNT/ML:::insulin, regular, human",2179743:::2179750:::311054:::415061:::1543200:::311057:::249026:::451439:::340286:::1544488:::311058:::1731314:::1543202:::343084:::249028:::340296:::343497:::1309342:::221109:::340327:::451437:::253182,Clinical Drug Form:::Clinical Drug:::Clinical Drug:::Clinical Drug:::Clinical Dose Group:::Clinical Drug:::Clinical Drug:::Clinical Drug Comp:::Clinical Drug Comp:::Clinical Drug:::Clinical Drug:::Clinical Drug Form:::Clinical Drug:::Clinical Drug Comp:::Clinical Drug:::Clinical Drug Comp:::Clinical Drug Comp:::Ingredient:::Ingredient:::Clinical Drug Comp:::Clinical Drug Comp:::Ingredient
3,Hydrochlo

In [16]:
light_model = LightPipeline(model)
light_result = light_model.fullAnnotate(sample_text)

er_vis = EntityResolverVisualizer()

er_vis.display(light_result[0],
               label_col='ner_chunk',
               resolution_col = 'rxnorm_code',
               document_col='document'
               )